In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import shuffle
from matplotlib import cm

%matplotlib inline

In [2]:
np.random.seed(101)
error_tab = []
lr_tab = []
acc_tab = []

In [3]:
df = pd.read_csv('tae.data', header=None)

In [4]:
columns = ['English speaker', 'Course', 'Course instructor', 'Summer or regular semester', 'Class size', 'Class attribute']
df.columns = columns

In [5]:
df.head()

,English speaker,Course,Course instructor,Summer or regular semester,Class size,Class attribute
0,1,23,3,1,19,3
1,2,15,3,1,17,3
2,1,23,3,2,49,3
3,1,5,2,2,33,3
4,2,7,11,2,55,3


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 6 columns):
English speaker               151 non-null int64
Course                        151 non-null int64
Course instructor             151 non-null int64
Summer or regular semester    151 non-null int64
Class size                    151 non-null int64
Class attribute               151 non-null int64
dtypes: int64(6)
memory usage: 7.1 KB


In [7]:
x = df.drop('Class attribute', axis=1).values
labels = df['Class attribute']
y = pd.get_dummies(df['Class attribute']).values

In [8]:
def layer_sizes(X, Y):
    n_x = 5 # input layer
    n_h1 = 50
    n_h2 = 20
    n_y = 3 # output layer
    return (n_x, n_h1, n_h2, n_y)
        
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
def initialize_parameters(n_x, n_h1, n_h2, n_y):
    W1 = np.random.randn(n_x,n_h1)*0.01
    b1 = np.zeros(n_h1)

    W2 = np.random.randn(n_h1,n_h2)*0.01
    b2 = np.zeros(n_h2)

    W3 = np.random.randn(n_h2,n_y)*0.01
    b3 = np.zeros(n_y)

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [ ]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    # Layer 1
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    # Layer 2
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    
    # Layer 3
    Z3 = np.dot(A2, W3) + b3
    A3 = sigmoid(Z3)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2,
             "Z3": Z3,
             "A3": A3}
    
    return A3, cache

In [ ]:
def compute_cost(A3, Y, parameters):
    m = Y.shape[0] # number of examples
    # cross entropy
    logprobs = np.multiply(np.log(A3), Y) + np.multiply((1 - Y), np.log(1 - A3))
    cost = - np.sum(logprobs) / m    
    
    cost = np.squeeze(cost)
    
    return cost

In [ ]:
def accuracy(output_vec):
    equals = np.equal(np.argmax(y, axis=1), np.argmax(output_vec, axis=1))
    acc = np.mean(equals)
    return acc

In [ ]:
def adapt_learning_rate(learning_rate, xi_d, xi_i, er_r, cost, prev_cost):
    if cost > er_r*prev_cost:
        learning_rate*=xi_d
    elif cost < prev_cost:
        learning_rate*=xi_i
    prev_cost = cost
    return learning_rate

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    
    #  m = X.shape[0]
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    A3 = cache['A3']
    
    Z1 = cache['Z1']
    Z2 = cache['Z2']
    '''
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ3 = A3 - Y
    dW3 = (1 / m) * np.dot(A2.T, dZ3)
    db3 = (1 / m) * np.sum(dZ3, axis=1, keepdims=True)
    
    dZ2 = np.multiply(np.dot(dZ3, W3.T), 1 - np.power(A2, 2))
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    
    dZ1 = np.multiply(np.dot(dZ2, W2.T), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(X.T, dZ1)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    '''
    
    ########## Layer 3
    delta3 = A3-Y
    dW3 = np.dot(A2.T, delta3)
    db3 = delta3
    
    ########## Layer 2
    delta2 = np.dot(delta3, W3.T) * sigmoid_prime(Z2)
    dW2 = np.dot(A1.T, delta2)
    db2 = delta2
    
    ########## Layer 1
    delta1 = np.dot(delta2, W2.T) * sigmoid_prime(Z1)
    dW1 = np.dot(X.T, delta1)
    db1 = delta1
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2,       
             "dW3": dW3,
             "db3": db3}
    return grads

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    dW3 = grads['dW3']
    db3 = grads['db3']

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1.sum(axis=0)

    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2.sum(axis=0)
    
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3.sum(axis=0)
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [ ]:
def nn_model(X, Y, n_h1, n_h2, xi_d, xi_i, er_r, learning_rate = 0.001, num_iterations = 5000, print_cost=True):
    prev_cost = 1000.0
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[3]
    
    parameters = initialize_parameters(n_x, n_h1, n_h2, n_y)
 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    max_acc = 0;
    
    for epoch in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A3, cache = forward_propagation(X, parameters)

        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A3, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Adaptive learning rate @@@ not decrement!!
        # learning_rate *= 1/(1 + learning_rate/(epoch+1) * epoch)
        learning_rate = adapt_learning_rate(learning_rate, xi_d, xi_i, er_r, cost, prev_cost)
        # learning_rate = 0.01
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate)

        if epoch % 500 == 0:
            #loss = -np.sum(y * np.log(A3))
            #error_tab.append(cost)
            lr_tab.append(learning_rate)
            equals = np.equal(np.argmax(y, axis=1), np.argmax(A3, axis=1))
            acc = np.mean(equals)
            # acc_tab.append(acc)
            print("Cost after iteration %i: %f, accuracy: %f" %(epoch, cost, acc))

    return parameters, A3

In [ ]:
def predict(parameters, X):  
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A3, cache = forward_propagation(X,parameters)
    predictions = A3 # > 0.5  
    return predictions

In [ ]:
# Build a model with a n_h-dimensional hidden layer
# parameters = nn_model(x, y, n_h1 = 60, n_h2 = 20, learning_rate = 0.01, num_iterations = 10000, print_cost=True)

# plot_decision_boundary(lambda x: predict(parameters, x.T), x, y)
# plt.title("Decision Boundary for hidden layer size " + str(4));

In [ ]:
# accuracy plot
# plt.plot(range(len(error_tab)), error_tab)

# print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

In [ ]:
# Create a function called "chunks" with two arguments, l and n:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [ ]:
# plt.plot(range(len(acc_tab)), acc_tab)

In [ ]:
n_number = list(range(40, 60, 10))
acc_array = []
for n_h1 in n_number:
    for n_h2 in n_number:
        parameters, A3 = nn_model(x, y, n_h1, n_h2, xi_d=0.7 , xi_i=1.05, er_r=1.04, num_iterations = 5000)
        acc = accuracy(output_vec = A3)
        acc_array.append(acc)
        print("1st layer: {} 2nd layer: {} Accuracy: {}".format(n_h1, n_h2, acc))

Cost after iteration 0: 2.073858, accuracy: 0.344371


c:\users\fryderyk\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
c:\users\fryderyk\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.
c:\users\fryderyk\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


Cost after iteration 500: nan, accuracy: 0.331126
Cost after iteration 1000: nan, accuracy: 0.331126
Cost after iteration 1500: nan, accuracy: 0.331126
Cost after iteration 2000: nan, accuracy: 0.324503
Cost after iteration 2500: nan, accuracy: 0.324503
Cost after iteration 3000: nan, accuracy: 0.344371
Cost after iteration 3500: nan, accuracy: 0.331126
Cost after iteration 4000: nan, accuracy: 0.324503
Cost after iteration 4500: nan, accuracy: 0.324503
1st layer: 40 2nd layer: 40 Accuracy: 0.32450331125827814
Cost after iteration 0: 2.087851, accuracy: 0.331126
Cost after iteration 500: nan, accuracy: 0.324503
Cost after iteration 1000: nan, accuracy: 0.324503
Cost after iteration 1500: nan, accuracy: 0.324503
Cost after iteration 2000: nan, accuracy: 0.344371
Cost after iteration 2500: nan, accuracy: 0.324503
Cost after iteration 3000: nan, accuracy: 0.344371
Cost after iteration 3500: nan, accuracy: 0.331126
Cost after iteration 4000: nan, accuracy: 0.324503
Cost after iteration 450

In [ ]:
# deviding acc_array into chunks
acc_array = list(chunks(acc_array, len(n_number)))
acc_array = np.array(acc_array)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(n_number, n_number, acc_array, rstride=1, cstride=1, cmap=cm.coolwarm,
    linewidth=0, antialiased=False)

In [ ]:
print(np.amax(acc_array))

In [ ]:
plt.plot(range(len(lr_tab)), lr_tab)